In [1]:
import random
import numpy as np

# Define the puzzle grid size
GRID_SIZE = 4

# Define the target numbers
TARGET_NUMBERS = ["o", "r", "w", "d"]


def generate_individual(puzzle):
    # Generate a random individual (grid) while preserving the initial values from the puzzle
    individual = np.copy(puzzle)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if individual[i][j] == 0:
                individual[i][j] = random.choice(TARGET_NUMBERS)
    return individual


def fitness(individual, puzzle):
    # Calculate the fitness score of an individual
    score = 0

    # Check each row
    for row in individual:
        if len(set(row)) == GRID_SIZE:
            score += 1

    # Check each column
    for j in range(GRID_SIZE):
        column = [individual[i][j] for i in range(GRID_SIZE)]
        if len(set(column)) == GRID_SIZE:
            score += 1

    # Check each 3x3 sub-grid
    for i in range(0, GRID_SIZE, 3):
        for j in range(0, GRID_SIZE, 3):
            subgrid = individual[i:i+3, j:j+3].flatten()
            if len(set(subgrid)) == GRID_SIZE:
                score += 1

    # Preserve the initial values from the puzzle
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if puzzle[i][j] != 0:
                if individual[i][j] == puzzle[i][j]:
                    score += 1

    return score


def crossover(parent1, parent2):
    # Perform crossover between two parents to produce two offspring
    point1 = random.randint(1, GRID_SIZE - 2)
    point2 = random.randint(1, GRID_SIZE - 2)
    offspring1 = np.copy(parent1)
    offspring2 = np.copy(parent2)
    offspring1[:point1], offspring1[point1:] = parent2[:point1], parent1[point1:]
    offspring2[:point2], offspring2[point2:] = parent1[:point2], parent2[point2:]
    return offspring1, offspring2


def mutate(individual, puzzle):
    # Perform mutation on an individual while preserving the initial values from the puzzle
    mutated_individual = np.copy(individual)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if puzzle[i][j] == "" and random.random() < 0.1:  # Mutation rate: 10% for empty cells only
                mutated_individual[i][j] = random.choice(TARGET_NUMBERS)
    return mutated_individual


def genetic_algorithm(puzzle, population_size, generations):
    # Initialize the population
    population = [generate_individual(puzzle) for _ in range(population_size)]

    for _ in range(generations):
        # Evaluate the fitness of each individual
        fitness_scores = [fitness(individual, puzzle) for individual in population]

        # Select parents for reproduction
        parents = random.choices(population, weights=fitness_scores, k=population_size)

        # Create the next generation
        next_generation = []
        for i in range(0, population_size, 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            offspring1, offspring2 = crossover(parent1, parent2)
            next_generation.extend([mutate(offspring1, puzzle), mutate(offspring2, puzzle)])

        # Replace the current population with the next generation
        population = next_generation

    # Return the best individual (solution)
    best_individual = max(population, key=lambda x: fitness(x, puzzle))
    return best_individual

def is_valid(best_individual):
    # Check if the matrix is valid
    for i in range(GRID_SIZE):
        if len(set(best_individual[i])) != GRID_SIZE:
            return False
        if len(set(best_individual[:, i])) != GRID_SIZE:
            return False
    return True

def generate_valid_matrix():
    # Generate a valid matrix
    while True:
        matrix = generate_matrix()
        if is_valid(matrix):
            return matrix

# Define the Sudoku puzzle
puzzle = np.array([
    ["", "", "r", "o"],
    ["o", "", "", "d"],
    ["r", "o", "", "w"],
    ["d", "w", "o", "r"]
])


# Solve the Sudoku puzzle
solution = genetic_algorithm(puzzle, population_size=100, generations=1000)

# Print the solution
print("Solution:")
print(solution)

Solution:
[['w' 'w' 'r' 'o']
 ['o' 'r' 'w' 'd']
 ['r' 'o' 'd' 'w']
 ['d' 'w' 'o' 'r']]
